In [ ]:
from google.colab import drive
drive.mount("/content/gdrive", force_remount=True)

In [ ]:
%ls /content/gdrive/My\ Drive/Colab\ Notebooks/Digital\ Image\ Processiing/HW6/*.py
import sys
sys.path.append('/content/gdrive/My Drive/Colab Notebooks/Digital Image Processiing/HW6')

In [ ]:
# import datasets
from keras.datasets import fashion_mnist

In [ ]:
# import base python libraries
import pandas as pd
import numpy as np
from numpy import save
from numpy import load
import matplotlib.pyplot as plt
import warnings, gc
import time

In [ ]:
# import ml framework libraries
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
import xgboost as xgb

In [ ]:
# import sklearn libs
from sklearn import datasets
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.model_selection import train_test_split
from skimage.measure import block_reduce
from skimage.util import view_as_windows
from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV
import pickle
#from xgboost import XGBClassifier

In [ ]:
# special imports
from pixelhop import Pixelhop

In [ ]:
np.random.seed(1)

# Preprocess
N_Train_Reduced = 10000    # 10000
N_Train_Full = 60000       # 50000
N_Test = 10000             # 10000

BS = 2000 # batch size

In [ ]:
def shuffle_data(X, y):
    shuffle_idx = np.random.permutation(y.size)
    X = X[shuffle_idx]
    y = y[shuffle_idx]
    return X, y

In [ ]:
def select_balanced_subset(images, labels, use_num_images):
    '''
    select equal number of images from each classes
    '''
    num_total, H, W, C = images.shape
    num_class = np.unique(labels).size
    num_per_class = int(use_num_images / num_class)

    # Shuffle
    images, labels = shuffle_data(images, labels)

    selected_images = np.zeros((use_num_images, H, W, C))
    selected_labels = np.zeros(use_num_images)

    for i in range(num_class):
        selected_images[i * num_per_class:(i + 1) * num_per_class] = images[labels == i][:num_per_class]
        selected_labels[i * num_per_class:(i + 1) * num_per_class] = np.ones((num_per_class)) * i

    # Shuffle again
    selected_images, selected_labels = shuffle_data(selected_images, selected_labels)
    print(selected_labels)

    return selected_images, selected_labels


In [ ]:
def Shrink(X, shrinkArg):
    #---- max pooling----
    pool = shrinkArg['pool']

    ch = X.shape[-1]  # get the number of channels

     # fill in the rest of max pooling
    if (pool > 1):
      X = block_reduce(X, block_size = (1, pool , pool, ch), func = np.max)

    #---- neighborhood construction
    win = shrinkArg['win']
    stride = shrinkArg['stride']
    pad = shrinkArg['pad']
    
    # fill in the rest of neighborhood construction
   
    if pad > 0:
      X = np.pad(X, ((0,0), (pad, pad), (pad, pad), (0, 0)), 'reflect')
    X = view_as_windows(X, (1, win , win , 1 ), (1, stride, stride, 1))
    X = X.reshape(X.shape[0], X.shape[1], X.shape[2], -1)

    print(X.shape[0], X.shape[1], X.shape[2], -1)
    return  X


In [ ]:
# example callback function for how to concate features from different hops
def Concat(X, concatArg):
    return X

In [ ]:
def get_feat(p2, X, num_layers=3):
    output = p2.transform_singleHop(X,layer=0)
    if num_layers>1:
        for i in range(num_layers-1):
            output = p2.transform_singleHop(output, layer=i+1)
    return output

In [ ]:
warnings.filterwarnings("ignore")
# ---------- Load MNIST data and split ----------
(x_train, y_train), (x_test,y_test) = fashion_mnist.load_data()

In [ ]:
# -----------Data Preprocessing-----------
x_train = np.asarray(x_train,dtype='float32')[:,:,:,np.newaxis]
x_test = np.asarray(x_test,dtype='float32')[:,:,:,np.newaxis]
y_train = np.asarray(y_train,dtype='int')
y_test = np.asarray(y_test,dtype='int')

# if use only 10000 images train pixelhop
x_train_reduced, y_train_reduced = select_balanced_subset(x_train, y_train, use_num_images=N_Train_Reduced)

x_train /= 255.0
x_test /= 255.0

x_train_reduced /=255.0
#x_test /=255.0

print(" input feature full train dataset shape: %s"%str(x_train.shape))
print(" input feature full test dataset shape: %s"%str(x_test.shape))

print(" input feature reduced train dataset shape: %s"%str(x_train_reduced.shape))
print(" input feature reduced test dataset shape: %s"%str(y_train_reduced.shape))
print(" y_train shape: %s"%str(y_train_reduced))

In [ ]:
# -----------Module 1: set PixelHop parameters-----------
module1_start_time = time.time()

# fill in this part
shrinkArgs = [{'func': Shrink, 'win':5, 'stride': 1, 'pad':2, 'pool': 1},
              {'func': Shrink, 'win':5, 'stride': 1, 'pad':0, 'pool': 2 }, 
              {'func': Shrink, 'win':5, 'stride': 1, 'pad':0, 'pool': 2}]
          
          
SaabArgs = [{'num_AC_kernels':-1, 'needBias':False, 'cw': False},
            {'num_AC_kernels':-1, 'needBias':True,  'cw': False}, 
            {'num_AC_kernels':-1, 'needBias':True,  'cw': False}]
        

concatArg = {'func':Concat}

In [ ]:
# -----------Module 1: Train PixelHop -----------
# TODO: fill in this part
pixelplus = Pixelhop(depth = 3,
                    TH1 = 0.0005,  #will vary this part
                    TH2 = 0.001,
                    SaabArgs = SaabArgs,
                    shrinkArgs = shrinkArgs, 
                    concatArg = concatArg).fit(x_train_reduced)

print("Module 1 took {0} seconds.".format(time.time() - module1_start_time))

In [ ]:
# Get K1 , K2, K3
Hop1_Energy = pixelplus.Energy['Layer0']  # list of length = K1
Hop2_Energy = pixelplus.Energy['Layer1']  # list of length = K2
Hop3_Energy = pixelplus.Energy['Layer2']  # list of length  = K3

print("Hop1 Energy Shape :", Hop1_Energy[0].shape)
print("Hop2 Energy Shape :",Hop1_Energy[0].shape)
print("Hop3 Energy Shape :",Hop1_Energy[0].shape)

In [ ]:
# save  the Pixelhop model                    
#pickle.dump(pixelplus, open('/content/gdrive/My Drive/Colab Notebooks/Digital Image Processiing/HW6/pixelHop_4units.sav', 'wb'))

In [ ]:
# --------- Module 2: get only Hop 3 feature for both training set and testing set -----------
# you can get feature "batch wise" and concatenate them if your memory is restricted
# TODO: fill in this part
#train_hop1_feats = pixelplus.transform(x_train)
#train_hop3_feats = get_feat(pixelplus, x_train_reduced)
#test_hop3_feats = get_feat(pixelplus, x_test)

In [ ]:
# --------- Module 2: get only Hop 3 feature for both training set and testing set -----------
# you can get feature "batch wise" and concatenate them if your memory is restricted
# TODO: fill in this part

module2_start_time = time.time()

# apply transform to whole 60k train images in batches
op1 = get_feat(pixelplus, x_train[:10000,:,:])
op2 = get_feat(pixelplus, x_train[10000:20000,:,:])
op3 = get_feat(pixelplus, x_train[20000:30000,:,:])
op4 = get_feat(pixelplus, x_train[30000:40000,:,:])
op5 = get_feat(pixelplus, x_train[40000:50000,:,:])
op6 = get_feat(pixelplus, x_train[50000:60000,:,:])

# apply transform to 10k test images
test_hop3_feats = get_feat(pixelplus, x_test)

# get only hop3 features by concatenating the batched transforms
#train_hop3_feats = np.concatenate((op1[3],op2[3],op3[3],op4[3],op5[3], op6[3]))

In [ ]:
# get only hop3 features by concatenating the batched transforms
train_hop3_feats = np.concatenate((op1,op2,op3,op4,op5,op6))

In [ ]:
print( "Hop3 train features shape: ", train_hop3_feats.shape)
print("Hop3 test features shape: ", test_hop3_feats.shape) 

In [ ]:
# --------- Module 2: standardization of the Hop features
STD = np.std(train_hop3_feats, axis=0, keepdims=1)
train_hop3_feats = train_hop3_feats/STD
test_hop3_feats = test_hop3_feats/STD

print("Standardized vector shape: ", STD.shape) 
print( "Hop3 train features shape: ", train_hop3_feats.shape)
print("Hop3 test features shape: ", test_hop3_feats.shape) 
print("input feature train shape: %s"%str(y_train.shape)) 

In [ ]:
train_hop3_feats = np.squeeze(train_hop3_feats)
test_hop3_feats = np.squeeze(test_hop3_feats)

print( "Hop3 train features shape: ", train_hop3_feats.shape)
print("Hop3 test features shape: ", test_hop3_feats.shape) 

print("Module 2 took {0} seconds.".format(time.time() - module2_start_time))

In [ ]:
 #---------- Module 3: Train XGBoost classifier on hop3 feature ---------

module3_start_time = time.time()

tr_acc = []
te_acc = []

# multiclass classification, so use multi:softprob
clf = xgb.XGBClassifier(n_jobs=-1,
                     objective='multi:softprob',
                    # tree_method='gpu_hist', gpu_id=None,
                    max_depth=3,n_estimators=100,
                    min_child_weight=5,gamma=1,
                    subsample=0.8,learning_rate=1.0,
                    nthread=8,colsample_bytree=1.0)


In [ ]:
# TODO: fill in the rest and report accuracy
clf.fit(train_hop3_feats, y_train)

print("Module 3 took {0} seconds.".format(time.time() - module3_start_time))

In [ ]:
y_train_pred = clf.predict(train_hop3_feats)
train_predictions = [round(value) for value in y_train_pred]
# evaluate accuracy for test 
tr_acc = accuracy_score(y_train, train_predictions)
print("Train Accuracy: %.2f%%" % (tr_acc * 100.0))

In [ ]:
y_test_pred = clf.predict(test_hop3_feats)
test_predictions = [round(value) for value in y_test_pred]
# evaluate accuracy for test 
te_acc = accuracy_score(y_test, test_predictions)
print("Test Accuracy: %.2f%%" % (te_acc * 100.0))

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix
import seaborn as sns

cf_matrix = confusion_matrix(y_test, y_test_pred)
print(cf_matrix)
  
# heatmap here and see percentages of data represented in each quadrant
plt.subplots(figsize =(15,8))
sns.heatmap(cf_matrix/np.sum(cf_matrix), annot=True, fmt='.2%',cmap='Blues',linewidths=.6, xticklabels='auto')


In [ ]:


##### with batched 


objective='multi:softprob',
                    # tree_method='gpu_hist', gpu_id=None,
                    max_depth=3,n_estimators=100,
                    min_child_weight=5,gamma=1,
                    subsample=0.8,learning_rate=1.0,
                    nthread=8,colsample_bytree=1.0)

## 1
TH2 = 0.001
TH1 = 0.005
l_r = 1.0

Hop3 train features shape:  (60000, 9)
Hop3 test features shape:  (10000, 9)

Hop1 Energy Shape : (23,)
Hop2 Energy Shape : (23,)
Hop3 Energy Shape : (23,)

Module 1 took 13.90117073059082 seconds.
Module 2 took 42.53356671333313 seconds.
Module 3 took 38.52722501754761 seconds.

Train Accuracy: 85.34%
Test Accuracy: 77.06%

## 2
TH2 = 0.001
TH1 = 0.25
l_r = 1.0

Hop3 train features shape:  (60000, 9)
Hop3 test features shape:  (10000, 9)


Hop1 Energy Shape : (23,)
Hop2 Energy Shape : (23,)
Hop3 Energy Shape : (23,)

Module 1 took 13.694656133651733 seconds.
Module 2 took 42.759296894073486 seconds.
Module 3 took 64.41364765167236 seconds.

Train Accuracy: 85.34%
Test Accuracy: 77.06%

## 3
TH2 = 0.001
TH1 = 0.01
l_r = 1.0

Hop3 train features shape:  (60000, 9)
Hop3 test features shape:  (10000, 9)

Hop1 Energy Shape : (23,)
Hop2 Energy Shape : (23,)
Hop3 Energy Shape : (23,)

Module 1 took 14.366409540176392 seconds.
Module 2 took 44.701276540756226 seconds.
Module 3 took 39.44400668144226 seconds.

Train Accuracy: 85.34%
Test Accuracy: 77.06%

## 4
TH2 = 0.001
TH1 = 0.00001
l_r = 1.0

Hop1 Energy Shape : (23,)
Hop2 Energy Shape : (23,)
Hop3 Energy Shape : (23,)

Hop3 train features shape:  (60000, 9)
Hop3 test features shape:  (10000, 9)


Module 1 took 14.366409540176392 seconds.
Module 2 took 44.542786836624146 seconds.
Module 3 took 40.646803855895996 seconds.

Train Accuracy: 85.34%
Test Accuracy: 77.06%


## 5
TH2 = 0.001
TH1 = 0.0005
l_r = 1.0

Hop3 train features shape:  (60000, 9)
Hop3 test features shape:  (10000, 9)

Module 1 took 18.425812482833862 seconds.
Module 2 took 42.14961504936218 seconds.
Module 3 took 39.322232723236084 seconds.

Train Accuracy: 85.34%
Test Accuracy: 77.06%
